# DPML | Latency Replay

In this notebook, we investigate the reproducibility of transformation sequences captured by `dpml`.

## Load Dependencies

In [1]:
from lineage import LeBatch

from sibyl import *
from datasets import load_dataset

import time
from tqdm.notebook import tqdm

## Create Datasets

In [2]:
dataset = load_dataset("glue", "sst2", split="train[:100]")
dataset = dataset.rename_column('sentence', 'text')

Reusing dataset glue (C:\Users\fabri\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


## Replay Test

### Routine to be Tracked

In [8]:
scheduler = SibylTransformScheduler("sentiment", num_INV = 4, num_SIB = 0)

In [10]:
text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []

batch_size= 10

records = []
startTime = time.perf_counter()
for i in tqdm(range(0, len(label), batch_size)):
    text_batch = text[i:i+batch_size]
    label_batch = label[i:i+batch_size]
    batch = (text_batch, label_batch)
    for transform in scheduler.sample():
        print(transform)
        # batch = transform.transform_batch(batch)
        batch = LeBatch(batch).apply(transform.transform_batch)
    records.append(batch)
print('Elapsed time: {:6.3f} seconds'.format(time.perf_counter() - startTime))

  0%|          | 0/10 [00:00<?, ?it/s]

Elapsed time: 13.475 seconds


In [11]:
records

[[<LeRecord:
  	 text="disguise new ganoin from the larental faille ",
  	 target="0",
  	 le_attrs={'transformation_provenance': <TransformationProvenance: {(0, "{'class': 'ChangeNumber', 'multiplier': 0.2, 'replacement': None, 'return_metadata': True}"), (2, "{'class': 'RandomSwapQwerty', 'return_metadata': True}"), (1, "{'class': 'Demojify', 'exact_match_only': False, 'randomize': False, 'return_metadata': True}"), (3, "{'class': 'ChangeHyponym', 'num_to_replace': inf, 'return_metadata': True}")}>, 'prev': <LeRecord:
  	 text="hide new secretions from the larental units ",
  	 target="0",
  	 le_attrs={'transformation_provenance': <TransformationProvenance: {(0, "{'class': 'ChangeNumber', 'multiplier': 0.2, 'replacement': None, 'return_metadata': True}"), (2, "{'class': 'RandomSwapQwerty', 'return_metadata': True}"), (1, "{'class': 'Demojify', 'exact_match_only': False, 'randomize': False, 'return_metadata': True}")}>, 'prev': <LeRecord:
  	 text="hide new secretions from the parent